In [ ]:
import pandas as pd
import plotly.express as px

from risk_data import get_factor_data


In [ ]:
def get_event_window(
    df: pd.Series | pd.DataFrame,
    event_date: str | pd.Timestamp,
    before: int = 21,
    after: int = 63
) -> pd.Series | pd.DataFrame:
    """
    Select a window of rows from a date-indexed DataFrame around an event date.

    Parameters:
    ----------
    df : pd.DataFrame
        DataFrame with a DateTimeIndex, assumed sorted.
    event_date : str | pd.Timestamp
        The target event date.
    before : int
        Number of rows before the event date to include.
    after : int
        Number of rows after the event date to include.

    Returns:
    -------
    pd.DataFrame
        Sliced DataFrame with [before rows] before and [after rows] after the next available event date.
    """
    
    event_date = pd.Timestamp(event_date)

    # Find next available index position (bfill means ≥ event_date)
    event_idx = df.index.get_indexer([event_date], method='bfill')[0]

    if event_idx == -1:
        raise ValueError(f"No date on or after {event_date} found in index!")

    start_idx = max(event_idx - before, 0)
    end_idx = event_idx + after + 1  # +1 because iloc is exclusive on end

    return df.iloc[start_idx:end_idx]


def run_event_study(returns_df: pd.Series | pd.DataFrame, 
                event_list: list[tuple[str, pd.Timestamp]], 
                before: int = 21, 
                after: int = 63) -> tuple:
    """
    Perform event study using specific factor names tied to events.

    Parameters:
    - returns_df: pd.DataFrame (date index, return columns)
    - event_list: list of (factor_name, event_date)
    - before: days before event
    - after: days after event

    Returns:

    """
    # offsets = range(-before, after + 1)
    # event_windows = []

    _list = []
    for factor_name, event_date in event_list:
        _list.append(
            get_event_window(returns_df[factor_name],
                             event_date, 
                             before=before, 
                             after=after)
            .rename('returns')
            .to_frame()
            # .assign(day_offset=range(-before, after + 1)) # THIS MIGHT BE TOO LONG
            .assign(day_offset=lambda df_: range(-before, -before + len(df_)))
            .assign(factor_name=factor_name)
            .assign(event_date=event_date)
            .reset_index()
            .set_index('day_offset')
        )
    return pd.concat(_list)


In [ ]:
factor_data = get_factor_data()
ret = factor_data.ret.to_pandas()
event_list = [('SPY', '2018-03-01'),
              ('SPY', '2025-04-04'),]
before = 21*3
after = 21*12
factor_name, event_date = event_list[0]
event_study = run_event_study(ret, event_list, before=before, after=after)
# event_study.reset_index().pivot(index='day_offset', columns=('factor_name', 'event_date'), values='returns')
df_cum = event_study.reset_index().pivot(index='day_offset', columns='event_date', values='returns').cumsum()
# event_study -= event_study.loc[0]
df_cum -= df_cum.loc[0]
px.line(df_cum, template='plotly_white', title='Cumulative Returns Around Events')